## Config Variables and Load CSV Data

In [1]:
import csv

image_data_path = 'Data/data_right_turn_improvement'

csv_data = []
with open(image_data_path + '/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        csv_data.append(row)

## Load Images

### Function to read an pre-process image

In [2]:
import numpy as np
import cv2
import sklearn

''' Function to pasrse source path and load image '''

def image_loader(path):
    filename = path.split('/')[-1]
    current_path = image_data_path + '/IMG/' + filename
    image = cv2.imread(current_path)

    return image


''' Function to loop through CSV and load a list of images and measurements '''

def csv_loader(csv_data, image_index=0, correction = 0.0, measurement_index=3):    
    images = []
    measurements = []

    for row in csv_data:
        source_path = row[image_index]
        measurement = float(row[measurement_index]) + correction

        images.append(image_loader(source_path))        
        measurements.append(measurement)
            
    assert len(images) == len(measurements)
    return images, measurements
    

''' Function that read and return a list of flipped images and measurements '''

def flip_images(image_list, measurement_list):
    n_images = len(image_list)
    n_measurements = len(measurement_list)
    assert n_images == n_measurements
    
    flipped_images = []
    flipped_measurements = []
    for i in range(n_images):
        flipped_images.append(np.fliplr(image_list[i]))
        flipped_measurements.append(-measurement_list[i])
        
    return flipped_images, flipped_measurements

### Load the Images from the front camera

In [3]:
images, measurements = csv_loader(csv_data)

n_center_images = len(images)

### The 2 cells bellow can load the images for the left and right camera

Left Camera

In [4]:
images_left, measurements_left = csv_loader(csv_data, image_index=1, correction= 0.2)

images.extend(images_left)
measurements.extend(measurements_left)

Right Camera

In [5]:
images_right, measurements_right = csv_loader(csv_data, image_index=2, correction= -0.2)

images.extend(images_right)
measurements.extend(measurements_right)

#### Images shape

In [6]:
image_shape = images[0].shape
n_images = len(images)
print('Number of images:', n_images)
print('Image shape:',image_shape)

Number of images: 82116
Image shape: (160, 320, 3)


Image sample (Center/Left/Right)

### Run this cell to enhance the set with flipped images

In [7]:
print('Number of images:', len(images))

Number of images: 82116


Sample (Normal/Flipped)

#### Load X and y train

In [8]:
X_train = np.array(images)
y_train = np.array(measurements)

## NVIDIA Network Model

In [11]:
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D, Dropout, Activation
from keras.models import Sequential
from keras.models import Model

model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=image_shape))
model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=20)

model.save('nvidia_center6.h5py')
print('DONE!')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`


Train on 65692 samples, validate on 16424 samples
Epoch 1/20
65692/65692 [==============================] - 40s - loss: 0.0152 - val_loss: 0.0358
Epoch 2/20
65692/65692 [==============================] - 39s - loss: 0.0135 - val_loss: 0.0325
Epoch 3/20
65692/65692 [==============================] - 38s - loss: 0.0130 - val_loss: 0.0341
Epoch 4/20
65692/65692 [==============================] - 39s - loss: 0.0128 - val_loss: 0.0459
Epoch 5/20
65692/65692 [==============================] - 39s - loss: 0.0127 - val_loss: 0.0376
Epoch 6/20
65692/65692 [==============================] - 39s - loss: 0.0126 - val_loss: 0.0355
Epoch 7/20
65692/65692 [==============================] - 39s - loss: 0.0125 - val_loss: 0.0392
Epoch 8/20
65692/65692 [==============================] - 39s - loss: 0.0124 - val_loss: 0.0323
Epoch 9/20
65692/65692 [==============================] - 39s - loss: 0.0122 - val_loss: 0.0355
Epoch 10/20
65692/65692 [==============================] - 39s - loss: 0.0122 - val_lo

In [10]:
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D, Dropout, Activation
from keras.models import Sequential
from keras.models import Model

model = Sequential()

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=image_shape))
model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(Convolution2D(64, 3, 3, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=20)

model.save('nvidia_center5.h5py')
print('DONE!')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="relu", strides=(2, 2))`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3)

Train on 65692 samples, validate on 16424 samples
Epoch 1/20
65692/65692 [==============================] - 34s - loss: 0.0137 - val_loss: 0.0270
Epoch 2/20
65692/65692 [==============================] - 33s - loss: 0.0118 - val_loss: 0.0277
Epoch 3/20
65692/65692 [==============================] - 34s - loss: 0.0110 - val_loss: 0.0151
Epoch 4/20
65692/65692 [==============================] - 33s - loss: 0.0101 - val_loss: 0.0241
Epoch 5/20
65692/65692 [==============================] - 33s - loss: 0.0094 - val_loss: 0.0219
Epoch 6/20
65692/65692 [==============================] - 34s - loss: 0.0089 - val_loss: 0.0182
Epoch 7/20
65692/65692 [==============================] - 34s - loss: 0.0084 - val_loss: 0.0189
Epoch 8/20
65692/65692 [==============================] - 34s - loss: 0.0080 - val_loss: 0.0167
Epoch 9/20
65692/65692 [==============================] - 34s - loss: 0.0077 - val_loss: 0.0166
Epoch 10/20
65692/65692 [==============================] - 34s - loss: 0.0074 - val_lo

In [11]:
from keras.layers import Flatten, Dense, Lambda, Convolution2D, MaxPooling2D, Cropping2D, Dropout, Activation
from keras.models import Sequential
from keras.models import Model

model = Sequential()
# model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=image_shape))
# model.add(Lambda(lambda x: (x / 255.0) - 0.5))

model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=image_shape))
model.add(Cropping2D(cropping=((70,25), (0,0))))

model.add(Convolution2D(24, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(36, 5, 5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(48, 5, 5, subsample=(2,2)))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Dropout(0.5))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=20)

model.save('nvidia_center4.h5py')
print('DONE!')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), strides=(2, 2), activation="relu")`
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), strides=(2, 2), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3))`


Train on 65692 samples, validate on 16424 samples
Epoch 1/20
65692/65692 [==============================] - 35s - loss: 0.0144 - val_loss: 0.0271
Epoch 2/20
65692/65692 [==============================] - 34s - loss: 0.0125 - val_loss: 0.0297
Epoch 3/20
65692/65692 [==============================] - 35s - loss: 0.0119 - val_loss: 0.0346
Epoch 4/20
65692/65692 [==============================] - 35s - loss: 0.0116 - val_loss: 0.0273
Epoch 5/20
65692/65692 [==============================] - 36s - loss: 0.0112 - val_loss: 0.0275
Epoch 6/20
65692/65692 [==============================] - 34s - loss: 0.0108 - val_loss: 0.0304
Epoch 7/20
65692/65692 [==============================] - 34s - loss: 0.0105 - val_loss: 0.0183
Epoch 8/20
65692/65692 [==============================] - 35s - loss: 0.0103 - val_loss: 0.0198
Epoch 9/20
65692/65692 [==============================] - 35s - loss: 0.0100 - val_loss: 0.0223
Epoch 10/20
65692/65692 [==============================] - 35s - loss: 0.0097 - val_lo